In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import logging
import sys
import pandas as pd
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.llms.openai import OpenAI
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

/Users/lbokeria/miniforge3/envs/reginald/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set up Azure GPT4

In [4]:
# Set up the LLM
openai_azure_api_key = os.environ["OPENAI_AZURE_API_KEY"]
azure_endpoint = "https://reginald-uk-south.openai.azure.com/"
api_version = "2024-02-01"

azure_gpt4 = AzureOpenAI(
    model="gpt-4",
    deployment_name="reginald-gpt4",
    api_key=openai_azure_api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [5]:
# Setup settings for vectorisation later
from reginald.models.models.llama_index import setup_settings
from reginald.models.setup_llm import DEFAULT_ARGS
from transformers import AutoTokenizer

from reginald.models.models.llama_index import (
    setup_settings,
    LlamaIndexLlamaCPP,
    set_global_tokenizer,
    compute_default_chunk_size,
)

/Users/lbokeria/miniforge3/envs/reginald/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [6]:
# set up settings
chunk_size = compute_default_chunk_size(
    max_input_size=4096, k=3
)  # calculate chunk size

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf"
).encode

set_global_tokenizer(tokenizer)

settings = setup_settings(
    llm                 = azure_gpt4,
    max_input_size      = DEFAULT_ARGS["max_input_size"],
    num_output          = DEFAULT_ARGS["num_output"],
    chunk_size          = chunk_size,
    chunk_overlap_ratio = DEFAULT_ARGS["chunk_overlap_ratio"],
    k                   = DEFAULT_ARGS["k"],
    tokenizer           = tokenizer,
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
Use pytorch device_name: mps
INFO:root:Settings llm: callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x30a148b90> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x31f2b8a40> completion_to_prompt=<function default_completion_to_prompt at 0x31f32b420> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None model='gpt-4' temperature=0.1 max_tokens=None logprobs=None top_logprobs=0 additional_kwargs={} max_retries=3 timeout=60.0 default_headers=None reuse_client=True api_key='23d8d18490684b5aa0c62aaf53ece404' api_base='https://api.openai.com/v1' api_version='2024-02-01' engine='reginald-gpt4' azure_endpoint='https://

## Personal OpenAI

In [7]:
# Try personal OpenAI
# import os
# import openai
# # os.environ["OPENAI_API_KEY"] = ""
# openai.api_key = os.environ["OPENAI_API_KEY"]

KeyError: 'OPENAI_API_KEY'

## Download the data

In [10]:
!mkdir -p '../../data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O '../../data/paul_graham/paul_graham_essay.txt'

--2024-06-12 13:43:21--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘../../data/paul_graham/paul_graham_essay.txt’

../../data/paul_gra 100%[===================>]  73.28K  --.-KB/s    in 0.004s  

2024-06-12 13:43:21 (16.0 MB/s) - ‘../../data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [8]:
reader = SimpleDirectoryReader("../../data/paul_graham/")
documents = reader.load_data()

ValueError: Directory ../../data/paul_graham/ does not exist.

## Generate the questions

In [ ]:
data_generator = DatasetGenerator.from_documents(documents)

In [ ]:
eval_questions = data_generator.generate_questions_from_nodes(num=2)

In [ ]:
eval_questions

In [ ]:
# gpt-4
# gpt4 = OpenAI(temperature=0, model="gpt-4")
evaluator_gpt4 = RelevancyEvaluator(llm=azure_gpt4)

In [ ]:
# create vector index
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True)

In [ ]:
# define jupyter display function
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": (
                response.source_nodes[0].node.get_content()[:1000] + "..."
            ),
            "Evaluation Result": eval_result.passing,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [ ]:
query_engine    = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[1])

In [ ]:
eval_result     = evaluator_gpt4.evaluate_response(
    query=eval_questions[1], response=response_vector
)

In [ ]:
display_eval_df(eval_questions[1], response_vector, eval_result)

In [ ]:
eval_result

In [ ]:
vars(eval_result)

In [ ]:
eval_result.contexts[1]

In [ ]:
import textwrap

wrapped_text = textwrap.fill(text, width=80) #text is the object which you want to print
print(wrapped_text)

In [ ]:
print(response_vector.source_nodes[0].node.get_content())

In [ ]:
print(response_vector.source_nodes[1].node.get_content())